<a href="https://colab.research.google.com/github/space-owner/Tensorflow-2/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Word2Vec

In [5]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
assert tf.__version__[0] > "1"

In [6]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [11]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split(" "))
print(">>> tokens length = ", len(tokens))
print(">>> tokens =", tokens)

>>> tokens length =  8
>>> tokens = ['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


In [12]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(">>> vocab =", vocab)

>>> vocab = {'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [13]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(">>> inverse_vocab =", inverse_vocab)

>>> inverse_vocab = {0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [14]:
example_sequence = [vocab[word] for word in tokens]
print(">>> example sequence =", example_sequence)

>>> example sequence = [1, 2, 3, 4, 5, 1, 6, 7]


In [16]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0)
print(">>> length of positive skip grams =", len(positive_skip_grams))

>>> length of positive skip grams = 26


In [18]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(1, 7): (the, sun)
(4, 2): (shimmered, wide)
(6, 1): (hot, the)
(4, 1): (shimmered, the)
(7, 6): (sun, hot)


In [23]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    seed=SEED,
    name="negative_sampling"
)
print(">>> negative_sampling_candidates =", negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

>>> negative_sampling_candidates = tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
['<pad>', 'the', 'wide', 'road']


In [24]:
print(">>> negative_sampling_candidates =", negative_sampling_candidates)

>>> negative_sampling_candidates = tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)


In [25]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

context = tf.concat([context_class, negative_sampling_candidates], 0)

label = tf.constant([1] + [0]*num_ns, dtype="int64")

target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 1
target_word     : the
context_indices : [7 0 1 2 3]
context_words   : ['sun', '<pad>', 'the', 'wide', 'road']
label           : [1 0 0 0 0]


In [26]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(1, shape=(), dtype=int32)
context : tf.Tensor([7 0 1 2 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [27]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(">> sampling_table =", sampling_table)

>> sampling_table = [0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
def generate_training_data(sequencesm window_size, num_ns, vocab_size, seed):
    return 0       